In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [3]:
# Hyperparameters
batch_size = 64
learning_rate = 1e-3
num_epochs = 20

# Transform: Normalize and Flatten Images
transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Lambda(lambda x: x.view(-1))  # Flatten the image
])

# Load CIFAR10 Dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Files already downloaded and verified
Files already downloaded and verified
cuda


<a id="3"></a> <br>
### Logistic Regression

In [11]:
# Create Logistic Regression Model
class LogisticRegressionModel(nn.Module):
    def __init__(self):
        super(LogisticRegressionModel, self).__init__()

        self.linear = nn.Linear(32 * 32 * 3, 10)

    def forward(self, x):
        out = self.linear(x.view(-1, 32 * 32 * 3))
        return out


model = LogisticRegressionModel().to(device)

error = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [12]:
# Traning the Model
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for (images, labels) in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = error(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    running_loss += loss.item()
        
    # Calculate Accuracy         
    correct = 0
    total = 0
    # Predict test dataset
    for images, labels in test_loader: 
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        predicted = torch.max(outputs.data, 1)[1]
        total += len(labels)
        correct += (predicted == labels).sum()
    
    accuracy = 100 * correct / float(total)
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch}/{num_epochs}], Loss: {avg_loss:.4f}, Acc: {accuracy:.4f}%")

Epoch [0/20], Loss: 0.0027, Acc: 29.5100%
Epoch [1/20], Loss: 0.0026, Acc: 32.1100%
Epoch [2/20], Loss: 0.0028, Acc: 33.3300%
Epoch [3/20], Loss: 0.0024, Acc: 34.7900%
Epoch [4/20], Loss: 0.0024, Acc: 35.3800%
Epoch [5/20], Loss: 0.0022, Acc: 35.9000%
Epoch [6/20], Loss: 0.0027, Acc: 36.0600%
Epoch [7/20], Loss: 0.0025, Acc: 36.5900%
Epoch [8/20], Loss: 0.0023, Acc: 36.2700%
Epoch [9/20], Loss: 0.0021, Acc: 36.8800%
Epoch [10/20], Loss: 0.0024, Acc: 36.4600%
Epoch [11/20], Loss: 0.0019, Acc: 37.5400%
Epoch [12/20], Loss: 0.0021, Acc: 37.5100%
Epoch [13/20], Loss: 0.0031, Acc: 36.9700%
Epoch [14/20], Loss: 0.0023, Acc: 37.8300%
Epoch [15/20], Loss: 0.0022, Acc: 37.9300%
Epoch [16/20], Loss: 0.0023, Acc: 37.8600%
Epoch [17/20], Loss: 0.0018, Acc: 38.3800%
Epoch [18/20], Loss: 0.0026, Acc: 38.5000%
Epoch [19/20], Loss: 0.0028, Acc: 38.0500%


In [9]:
# visualization
# plt.plot(iteration_list,loss_list)
# plt.xlabel("Number of iteration")
# plt.ylabel("Loss")
# plt.title("Logistic Regression: Loss vs Number of iteration")
# plt.show()

<a id="4"></a> <br>
### Artificial Neural Network (ANN)

In [9]:
# Create ANN Model
class ANNModel(nn.Module):
    def __init__(self):
        super(ANNModel, self).__init__()

        self.fc1 = nn.Linear(32 * 32 * 3, 128) 
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 32)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(32, 10)  
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        return out


model = ANNModel().to(device)
error = nn.CrossEntropyLoss()
learning_rate = 0.02
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [11]:
num_epochs = 50

# Traning the Model
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for (images, labels) in train_loader:
        images = images.view(-1, 3 * 32 * 32).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = error(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    running_loss += loss.item()
        
    # Calculate Accuracy         
    correct = 0
    total = 0
    # Predict test dataset
    for images, labels in test_loader: 
        images = images.view(-1, 3 * 32 * 32).to(device)
        labels = labels.to(device)

        outputs = model(images)
        predicted = torch.max(outputs.data, 1)[1]
        total += len(labels)
        correct += (predicted == labels).sum()
    
    accuracy = 100 * correct / float(total)
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch}/{num_epochs}], Loss: {avg_loss:.4f}, Acc: {accuracy:.4f}%")

Epoch [0/50], Loss: 0.0029, Acc: 23.0600%
Epoch [1/50], Loss: 0.0026, Acc: 27.9200%
Epoch [2/50], Loss: 0.0026, Acc: 34.3600%
Epoch [3/50], Loss: 0.0023, Acc: 36.7000%
Epoch [4/50], Loss: 0.0021, Acc: 37.9600%
Epoch [5/50], Loss: 0.0020, Acc: 38.9300%
Epoch [6/50], Loss: 0.0023, Acc: 40.8500%
Epoch [7/50], Loss: 0.0021, Acc: 43.3700%
Epoch [8/50], Loss: 0.0027, Acc: 42.9400%
Epoch [9/50], Loss: 0.0014, Acc: 44.1700%
Epoch [10/50], Loss: 0.0021, Acc: 38.7400%
Epoch [11/50], Loss: 0.0021, Acc: 42.2500%
Epoch [12/50], Loss: 0.0017, Acc: 45.3600%
Epoch [13/50], Loss: 0.0015, Acc: 48.1400%
Epoch [14/50], Loss: 0.0018, Acc: 44.9800%
Epoch [15/50], Loss: 0.0012, Acc: 47.8300%
Epoch [16/50], Loss: 0.0022, Acc: 45.0700%
Epoch [17/50], Loss: 0.0016, Acc: 47.7100%
Epoch [18/50], Loss: 0.0018, Acc: 43.0100%
Epoch [19/50], Loss: 0.0018, Acc: 43.1000%
Epoch [20/50], Loss: 0.0016, Acc: 46.5200%
Epoch [21/50], Loss: 0.0015, Acc: 47.7000%
Epoch [22/50], Loss: 0.0016, Acc: 47.7200%
Epoch [23/50], Loss: 

In [18]:
# # visualization loss 
# plt.plot(iteration_list,loss_list)
# plt.xlabel("Number of iteration")
# plt.ylabel("Loss")
# plt.title("ANN: Loss vs Number of iteration")
# plt.show()

# # visualization accuracy 
# plt.plot(iteration_list,accuracy_list,color = "red")
# plt.xlabel("Number of iteration")
# plt.ylabel("Accuracy")
# plt.title("ANN: Accuracy vs Number of iteration")
# plt.show()

<a id="5"></a> <br>
### Convolutional Neural Network (CNN)

In [16]:
# Create CNN Model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(32 * 5 * 5, 10) 
    
    def forward(self, x):
        out = self.cnn1(x)
        out = self.relu1(out)
        out = self.maxpool1(out)
        out = self.cnn2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        return out


model = CNNModel().to(device)
error = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [17]:
num_epochs = 50

# Traning the Model
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for (images, labels) in train_loader:
        images = images.to(device).view(-1, 3, 32, 32)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = error(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    running_loss += loss.item()
        
    # Calculate Accuracy         
    correct = 0
    total = 0
    # Predict test dataset
    for images, labels in test_loader: 
        images = images.to(device).view(-1, 3, 32, 32)
        labels = labels.to(device)

        outputs = model(images)
        predicted = torch.max(outputs.data, 1)[1]
        total += len(labels)
        correct += (predicted == labels).sum()
    
    accuracy = 100 * correct / float(total)
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch}/{num_epochs}], Loss: {avg_loss:.4f}, Acc: {accuracy:.4f}%")

Epoch [0/50], Loss: 0.0019, Acc: 49.4100%
Epoch [1/50], Loss: 0.0016, Acc: 56.0300%
Epoch [2/50], Loss: 0.0018, Acc: 57.9300%
Epoch [3/50], Loss: 0.0014, Acc: 60.7200%
Epoch [4/50], Loss: 0.0018, Acc: 60.5800%
Epoch [5/50], Loss: 0.0015, Acc: 63.3000%
Epoch [6/50], Loss: 0.0016, Acc: 63.1800%
Epoch [7/50], Loss: 0.0013, Acc: 64.0100%
Epoch [8/50], Loss: 0.0013, Acc: 65.8300%
Epoch [9/50], Loss: 0.0018, Acc: 64.2600%
Epoch [10/50], Loss: 0.0011, Acc: 65.7700%
Epoch [11/50], Loss: 0.0013, Acc: 65.0800%
Epoch [12/50], Loss: 0.0008, Acc: 66.4300%
Epoch [13/50], Loss: 0.0016, Acc: 66.2300%
Epoch [14/50], Loss: 0.0014, Acc: 67.7000%
Epoch [15/50], Loss: 0.0005, Acc: 66.9300%
Epoch [16/50], Loss: 0.0011, Acc: 67.5000%
Epoch [17/50], Loss: 0.0008, Acc: 65.6800%
Epoch [18/50], Loss: 0.0009, Acc: 68.0400%
Epoch [19/50], Loss: 0.0005, Acc: 67.4300%
Epoch [20/50], Loss: 0.0007, Acc: 68.8600%
Epoch [21/50], Loss: 0.0013, Acc: 68.5700%
Epoch [22/50], Loss: 0.0013, Acc: 66.5400%
Epoch [23/50], Loss: 

In [29]:
# # visualization loss 
# plt.plot(iteration_list,loss_list)
# plt.xlabel("Number of iteration")
# plt.ylabel("Loss")
# plt.title("CNN: Loss vs Number of iteration")
# plt.show()

# # visualization accuracy 
# plt.plot(iteration_list,accuracy_list,color = "red")
# plt.xlabel("Number of iteration")
# plt.ylabel("Accuracy")
# plt.title("CNN: Accuracy vs Number of iteration")
# plt.show()